In [1]:
import gym
import numpy as np
import torch as th
from ActiveCritic.metaworld.metaworld.envs import \
    ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from ActiveCritic.model_src.transformer import (CriticTransformer, ModelSetup,
                                                TransformerModel)
from ActiveCritic.model_src.whole_sequence_model import (
    WholeSequenceActor, WholeSequenceCritic, WholeSequenceModelSetup)
from ActiveCritic.policy.active_critic_policy import (ACPOptResult,
                                                      ActiveCriticPolicy,
                                                      ActiveCriticPolicySetup)
from ActiveCritic.tests.test_utils.utils import make_wsm_setup
from ActiveCritic.utils.gym_utils import (DummyExtractor, make_policy_dict,
                                          new_epoch_reach)
from ActiveCritic.utils.pytorch_utils import make_partially_observed_seq
from ActiveCritic.utils.gym_utils import make_dummy_vec_env

from gym.wrappers import TimeLimit
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv

th.manual_seed(0)

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
def make_obs_act_space(obs_dim, action_dim):
    obs_array_low = [0]*obs_dim
    obs_array_high = [1]*obs_dim
    action_low = [0]*action_dim
    action_high = [1]*action_dim
    observation_space = gym.spaces.box.Box(
        np.array(obs_array_low), np.array(obs_array_high), (obs_dim,), float)
    action_space = gym.spaces.box.Box(
        np.array(action_low), np.array(action_high), (action_dim,), float)
    return observation_space, action_space

In [3]:
seq_len = 4
d_output = 2
d_result = 1

wsm_actor_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=TransformerModel)
wsm_critic_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=CriticTransformer, d_result=d_result)
actor = WholeSequenceActor(wsm_actor_setup)
critic = WholeSequenceCritic(wsm_critic_setup)

In [4]:
obs_dim = 4
act_dim = 2

In [5]:
obs, acts = make_obs_act_space(obs_dim, act_dim)

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [6]:
def make_acps(seq_len, extractor, new_epoch):
    acps = ActiveCriticPolicySetup()
    acps.device='cuda'
    acps.epoch_len=seq_len
    acps.extractor=extractor
    acps.new_epoch=new_epoch
    acps.opt_steps=100
    acps.optimisation_threshold=0.5
    acps.inference_opt_lr = 1e-1
    acps.optimize = True
    return acps

acps = make_acps(seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch_reach)

In [7]:
ac = ActiveCriticPolicy(observation_space=obs, action_space=acts, actor=actor, critic=critic, acps=acps)

In [8]:
current_step = 2
batch_size = 2
org_actions = th.ones([batch_size,acps.epoch_len,act_dim], device=acps.device, dtype=th.float, requires_grad=True)
opt_actions = th.zeros([batch_size,acps.epoch_len,act_dim], device=acps.device, dtype=th.float, requires_grad=True)

pro_opt_actions = ac.proj_actions(org_actions, opt_actions, current_step=current_step)

assert th.equal(org_actions[:,:current_step], pro_opt_actions[:, :current_step])
assert th.equal(opt_actions[:,current_step:], pro_opt_actions[:, current_step:])



In [9]:
current_step = 0
org_actions = th.ones([batch_size,acps.epoch_len,act_dim], device=acps.device, dtype=th.float, requires_grad=False)
opt_actions = th.zeros([batch_size,acps.epoch_len,act_dim], device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2*th.ones([batch_size,current_step+1,obs_dim], device=acps.device, dtype=th.float, requires_grad=False)
org_obs_seq = 2*th.ones([batch_size,current_step+1,obs_dim], device=acps.device, dtype=th.float, requires_grad=False)
optimizer = th.optim.Adam([opt_actions], lr=1e-1)
goal_label = th.ones([batch_size], device=acps.device, dtype=th.float)

actions, critic_result = ac.inference_opt_step(org_actions=org_actions, opt_actions=opt_actions, obs_seq=obs_seq, optimizer=optimizer, goal_label=goal_label, current_step=current_step)

last_critic_result = th.clone(critic_result.detach())

In [10]:
for i in range(3):
    actions, critic_result = ac.inference_opt_step(org_actions=org_actions, opt_actions=opt_actions, obs_seq=obs_seq, optimizer=optimizer, goal_label=goal_label, current_step=current_step)
    print(critic_result)
    assert th.equal(org_actions[:,:current_step], actions[:, :current_step]), 'org_actions were overwritten'
    assert not th.equal(opt_actions[:,current_step:], org_actions[:, current_step:])
    assert th.all(critic_result > last_critic_result), 'optimisation does not work.'
    last_critic_result = th.clone(critic_result.detach())

tensor([[-0.2089],
        [-0.2089]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.1465],
        [-0.1465]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0714],
        [-0.0714]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [11]:
actions

tensor([[[-0.3926, -0.3934],
         [ 0.2593,  0.3866],
         [ 0.3968, -0.4009],
         [ 0.4011, -0.4001]],

        [[-0.3926, -0.3934],
         [ 0.2593,  0.3866],
         [ 0.3968, -0.4009],
         [ 0.4011, -0.4001]]], device='cuda:0', requires_grad=True)

In [12]:
actions, expected_success = ac.optimize_act_sequence(actions=actions, observations=obs_seq, current_step=current_step)
assert th.equal(org_actions[:,:current_step], actions[:, :current_step]), 'org_actions were overwritten'
assert not th.equal(opt_actions[:,current_step:], org_actions[:, current_step:])
assert th.all(expected_success >= ac.args_obj.optimisation_threshold), 'optimisation does not work.'
assert th.equal(obs_seq, org_obs_seq)


In [13]:
expected_success

tensor([[0.5048],
        [0.5048]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
obs_seq

tensor([[[2., 2., 2., 2.]],

        [[2., 2., 2., 2.]]], device='cuda:0')

In [15]:
org_obs_seq

tensor([[[2., 2., 2., 2.]],

        [[2., 2., 2., 2.]]], device='cuda:0')

In [16]:
from ActiveCritic.tests.test_utils.utils import make_wsm_setup, make_obs_act_space, make_acps
from ActiveCritic.utils.gym_utils import new_epoch_pap, DummyExtractor

import gym
import numpy as np
import torch as th
from ActiveCritic.metaworld.metaworld.envs import \
    ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from ActiveCritic.model_src.transformer import (CriticTransformer, ModelSetup,
                                                TransformerModel)
from ActiveCritic.model_src.whole_sequence_model import (
    WholeSequenceActor, WholeSequenceCritic, WholeSequenceModelSetup)
from ActiveCritic.policy.active_critic_policy import (ACPOptResult,
                                                      ActiveCriticPolicy,
                                                      ActiveCriticPolicySetup)
from ActiveCritic.tests.test_utils.utils import make_wsm_setup
from ActiveCritic.utils.gym_utils import (DummyExtractor, make_policy_dict,
                                          new_epoch_reach)
from ActiveCritic.utils.pytorch_utils import make_partially_observed_seq
from ActiveCritic.utils.gym_utils import make_dummy_vec_env

from gym.wrappers import TimeLimit
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv

th.manual_seed(0)
def setup_ac():
    seq_len = 4
    d_output = 2
    d_result = 1
    obs_dim = 3
    batch_size = 2
    wsm_actor_setup = make_wsm_setup(
        seq_len=seq_len, d_output=d_output, model_class=TransformerModel)
    wsm_critic_setup = make_wsm_setup(
        seq_len=seq_len, d_output=d_output, model_class=CriticTransformer, d_result=d_result)
    acps = make_acps(
        seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch_pap)
    obs_space, acts_space = make_obs_act_space(
        obs_dim=obs_dim, action_dim=d_output)
    actor = WholeSequenceActor(wsm_actor_setup)
    critic = WholeSequenceCritic(wsm_critic_setup)
    ac = ActiveCriticPolicy(observation_space=obs_space, action_space=acts_space,
                            actor=actor, critic=critic, acps=acps)
    return ac, acps, d_output, obs_dim, batch_size

In [17]:
th.manual_seed(0)

current_step = 1
ac, acps, act_dim, obs_dim, batch_size = setup_ac()


org_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2*th.ones([batch_size, acps.epoch_len, obs_dim],
                    device=acps.device, dtype=th.float, requires_grad=False)
org_obs_seq = 2*th.ones([batch_size, acps.epoch_len, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)

In [18]:
obs_seq.shape

torch.Size([2, 4, 3])

In [19]:
result = ac.forward(observation_seq=obs_seq, action_seq =org_actions, optimize=True, current_step=1)

In [20]:
dve, gt_policy = make_dummy_vec_env('reach', 10)
de = DummyExtractor()

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [104]:
def setup_ac_reach():
    seq_len = 50
    env, gt_policy = make_dummy_vec_env('reach', seq_len=seq_len)
    d_result = 1
    d_output = env.action_space.shape[0]
    wsm_actor_setup = make_wsm_setup(
        seq_len=seq_len, d_output=d_output, model_class=TransformerModel)
    wsm_critic_setup = make_wsm_setup(
        seq_len=seq_len, d_output=d_output, model_class=CriticTransformer, d_result=d_result)
    acps = make_acps(
        seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch_reach)
    actor = WholeSequenceActor(wsm_actor_setup)
    critic = WholeSequenceCritic(wsm_critic_setup)
    ac = ActiveCriticPolicy(observation_space=env.observation_space, action_space=env.action_space,
                            actor=actor, critic=critic, acps=acps)
    return ac, acps, env

In [114]:
th.manual_seed(0)
ac, acps, env = setup_ac_reach()

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [120]:
obsv = env.reset()
last_obsv = th.tensor(obsv)
all_taken_actions = []
all_observations = [obsv]
for i in range(50):
    action = ac.predict(obsv)
    all_taken_actions.append(action)
    obsv, rew, dones, info = env.step(action)
    all_observations.append(obsv)
    assert len(th.nonzero(ac.obs_seq[:,ac.current_step+1:])) == 0
    if new_epoch_reach(last_obsv, th.tensor(obsv)):
        assert ac.current_step == ac.args_obj.epoch_len - 1
        ata = th.tensor(all_taken_actions).transpose(0,1)
        print(th.equal(ata.to('cuda'), ac.current_result.gen_trj))
        aob = th.tensor(all_observations).transpose(0,1)[:,:50]
        print(th.equal(aob.to('cuda'), ac.obs_seq))
        assert ac.current_result.expected_succes_before < ac.current_result.expected_succes_after


True
True


In [59]:
ac.current_step

49

In [31]:
ac.args_obj.optimize

True

In [32]:
ac.args_obj.optimisation_threshold

0.9

In [33]:
ac.args_obj.opt_steps

100

In [34]:
ac.current_result.expected_succes_after

tensor([[1.0657]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [35]:
ac.current_result.expected_succes_before

tensor([[0.4587]], device='cuda:0', grad_fn=<AddmmBackward0>)